# Análisis del Plebiscito en Twitter a través de hashtags

A continuación se muestra a través de un sencillo ejercicio como separar hashtags para clasificar texto manualmente utilizando [Apache Spark](https://spark.apache.org). Para empezar es necesario cargar previamente los tweets que se encuentran en un archivo en este [repositorio GitHub](https://github.com/vramirez/plebiscito/tree/master/data). El archivo json comprimido con tweets capturados desde el 2 de Octubre (día de la votación) hasta las 5 p.m. del día siguiente.

A continuación instrucciones para convertir el archivo de tweets en una tabla y realizar el análisis en ([DataBricks Community Edition](https://community.cloud.databricks.com/)), una plataforma para utilizar Spark. En el caso de la edición *Community* es **gratis**.

Lo primero es crear el cluster. Como es una plataforma gratuita, Databricks Community Edition ofrece una máquina de 6GB en RAM, lo suficiente para análisis pequeños, prototipos y pruebas. La máquina se elimina cuando lleva una hora de inactividad.

Clusters --> Create Cluster --> Name --> Ok

Table --> Create Table --> File --> Seleccionar archivo con los tweets y esperar a que cargue

Una vez que haya terminado el cargue dar clic "Preview Table" y:



1. Table Name = "Plebiscito" 
2. File type = Json 
3. Clic en "Create Table"

En el archivo se encuentran más de 30 mil tweets

In [2]:
# import de las librerias necesarias
from pyspark.sql.functions import *

In [3]:
%sql cache table plebiscito --optimizacion de la tabla en memoria

In [4]:
%sql

select count(*) as `Total Tweets` from Plebiscito 

## Formato de fechas
Las fechas en el archivo json tal y como las entrega Twitter, vienen en un formato difícil de manejar. Es necesario transformarla a algo más entendible como YYYY-MM-DD HH24:mm:ss

In [6]:
%sql 
--Ejemplos de fechas
select created_at  `Formato nativo fechas json` from plebiscito where text is not null limit 7

## Cambio de formato de fecha
A continuación se da formato a la fecha del formato json de Twitter y se crea una nueva tabla (`Plebiscito2`) con los mismos datos de la tabla `Plebiscito` solo que se añade un campo llamado `time` el cual contiene el timestamp

In [8]:
content = sqlContext.sql("select *, cast ( from_unixtime( unix_timestamp(concat( substring(created_at,27,4),' ', substring(created_at,5,15)), 'yyyy MMM dd HH:mm:ss')-18000) as timestamp) time from Plebiscito where text is not null")
#Creacion de nueva tabla
content.registerTempTable("plebiscito2")

In [9]:
%sql cache table plebiscito2

In [10]:
#Comparación de los dos formatos
display(content.select(content.created_at.alias("Formato Fecha Antes"),content.time.alias("Formato Fecha Después")))


In [11]:
%sql 
--RANGOS DE FECHAS minimo y máximo de tweets capturados
select min(time) `Rango de Fechas` from plebiscito2 union select max(time) Fechas from plebiscito2;

## Clasificación de tweets por el Si y por el No
La clasificación fue muy sencilla. Simplemente los que decían #HoyVotoSi, #YoVotoSi, #SiALaPaz se clasificaron por el Si, mientras #SoyColombiaNO,#HoyVotoNo, #YoVotoNo se clasificaron por el No. Lo cual tiene muchos puntos en contra, ya que no se mide el sarcasmo, por ejemplo. Se eligieron estos hashtags porque se observaron entre las tendencias (Trending Topics).

In [13]:
%sql 
--Tweets representando el SI
select count(*) `Votos por el Si` from Plebiscito2 
where (lower(text) like lower('%#HoyVotoSi%') or lower(text) like lower('%#YoVotoSi%') or lower(text) like lower('%#SiALaPaz%'))

In [14]:
%sql 
--Tweets representando el NO
select count(*) `Votos por el No`  from Plebiscito2 
where (lower(text) like lower('%#VotoNo%') or lower(text) like lower('%#HoyVotoNo%') or lower(text) like lower('%#SoyColombiaNO%'))

## Datasets
Se crea el dataset `votepleb` en el cual simplemente están agrupados por el SI o por el NO, **según el hashtag**. Así mismo se crea `votepleb1` donde se separan por cada uno de los hashtags

In [16]:
votepleb = sqlContext.sql("select 'SI' as Tweets, dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#HoyVotoSi%') or lower(text) like lower('%#YoVotoSi%') or lower(text) like lower('%#SiALaPaz%') union all select 'NO' as Tweets,dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#VotoNo%') or lower(text) like lower('%#SoyColombiaNO%') or lower(text) like lower('%#HoyVotoNo%')")

votepleb1 = sqlContext.sql("select '#HoyVotoSi' as Tweets, dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#HoyVotoSi%') union all select '#YoVotoSi' as Tweets, dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#YoVotoSi%') union all select '#SiALaPaz' as Tweets, dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#SiALaPaz%')                          union all select '#VotoNo' as Tweets,dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#VotoNo%') union all select '#SoyColombiaNO' as Tweets,dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#SoyColombiaNO%') union all select '#HoyVotoNo' as Tweets,dayofmonth(time) dia,hour(time) Hora from Plebiscito2 where lower(text) like lower('%#HoyVotoNo%')")


In [17]:
display(votepleb.groupBy("tweets").count())

In [18]:
display(votepleb1.groupBy("tweets").count())

In [19]:
#Filtro de resultado para el dia 2 de Octubre (Domingo)
display(votepleb1.where("dia==2"))

In [20]:
#Filtro de resultado para el dia 2 de Octubre (Domingo)
display(votepleb1.where("dia==3"))

## División por dias
A continuación se separa el dataset `votepleb`que contiene los tweets clasificados manualmente como Si o No y se divide para el día de elecciones (domingo 2 de Octubre) y para el dia siguiente.

In [22]:
diaelecciones=votepleb.where("dia==2")
diaelecciones.count()
postday=votepleb.where("dia==3")
postday.count()

#Tendencias Plebiscito
Los tweets con los hashtags: #HoyVotoSi, #YoVotoSi, #SiALaPaz se clasificaron por el Si, mientras #SoyColombiaNO,#HoyVotoNo, #YoVotoNo se clasificaron por el No. Se eligieron estos hashtags porque se observaron entre las tendencias (Trending Topics). Esta clasificación simple tiene un punto en contra, y es que no se mide el sarcasmo o la ironía, por ejemplo.

#Lunes post-plebiscito
Finalmente se repite el mismo análisis para el lunes, post-plebiscito cuando la gente se seguía manifestando a favor y en contra. Aunque el número de tweets disminuyó drásticamente, la tendencia por el SI persistía.

##Detalle
A continuación se hace una disgregación de los tweets de acuerdo al hashtag .

In [26]:
display(diaelecciones.orderBy("tweets",asc=True))

In [27]:
display(diaelecciones)

In [28]:
postday.first()

In [29]:
display(postday)

In [30]:
display(postday)

##Conclusiones
Este análisis es bastante básico, pues los tweets se separan solamente por el hashtag, independiente de la intención del texto. Adicionalmente, no se capturaron otros hashtags que pudieron tener cantidades similares o hashtags mal escritos, por ejemplo. Este ejercicio es a manera de ejemplo de exploración y análisis de datos.